In [1]:
import share_topic as st
import scanpy as sc

In [2]:
rna =sc.read_h5ad('/data/nelkazwi/share-topic/lymphoma_data/rna_B_lympho.h5ad')
atac =sc.read_h5ad("/data/nelkazwi/share-topic/lymphoma_data/atac_B_lympho.h5ad")
gamma=1
tau=0.5
n_topics=5
n_samples=20
n_burnin=1
batch_size=500
alpha=50/n_topics
beta=0.1

In [4]:
st_obj = st.SHARE_topic(rna,atac,n_topics,alpha,beta,gamma,tau)
theta, lam, phi= st_obj.SHARE_Topic(batch_size,n_samples,n_burnin,dev= "cuda:0",save_data=False,path="")

Processing: 100%|██████████| 20/20 [01:51<00:00,  5.59s/it]
